### Aula: Introdução às Redes Neurais




## 1. O que são Redes Neurais?

Redes neurais artificiais são modelos computacionais inspirados no cérebro humano. Elas são utilizadas para reconhecer padrões e resolver problemas complexos. O conceito central de uma rede neural é o **neurônio artificial**, que recebe entradas, aplica pesos e uma função de ativação, e gera uma saída.

Uma rede neural pode ser representada matematicamente por:

$$ y = f \left( \sum_{i=1}^{n} w_i x_i + b \right) $$

onde:
- $ x_i $ são as entradas,
- $ w_i $ são os pesos associados a cada entrada,
- $ b $ é o viés (bias),
- $ f $ é a função de ativação,
- $ y $ é a saída do neurônio.

Ajustando os pesos e o viés, conseguimos treinar a rede para realizar tarefas específicas.



## 2. Arquitetura das Redes Neurais

Uma rede neural é composta por várias **camadas** de neurônios:

- **Camada de entrada**: recebe os dados de entrada.
- **Camadas ocultas**: aplicam transformações não lineares aos dados.
- **Camada de saída**: fornece a previsão do modelo.

O número de neurônios em cada camada e o tipo de função de ativação influenciam diretamente o desempenho da rede.

A equação geral de uma rede com múltiplas camadas é:

$$ a^{(l)} = f \left( W^{(l)} a^{(l-1)} + b^{(l)} \right) $$

onde:
- $ a^{(l)} $ é a ativação da camada $ l $,
- $ W^{(l)} $ é a matriz de pesos da camada $ l $,
- $ b^{(l)} $ é o vetor de viés da camada $ l $,
- $ f $ é a função de ativação.



## 3. Treinamento de uma Rede Neural

O treinamento de uma rede neural envolve três passos principais:

1. **Forward Propagation** (propagação para frente): os dados passam pelas camadas e a saída é calculada.
2. **Cálculo da Função de Custo**: mede a diferença entre a previsão da rede e o valor real esperado.
3. **Backpropagation** (propagação para trás): ajusta os pesos da rede para minimizar o erro.

A atualização dos pesos é feita usando o **Gradiente Descendente**, um algoritmo que ajusta os pesos iterativamente para minimizar a função de custo:

$$ w = w - \alpha \frac{\partial J}{\partial w} $$

onde:
- $ \alpha $ é a taxa de aprendizado,
- $ J $ é a função de custo,
- $ \frac{\partial J}{\partial w} $ é o gradiente da função de custo em relação ao peso $ w $.


## 4. Funções de Ativação

As funções de ativação introduzem não-linearidade nas redes neurais. As mais comuns são:

- **ReLU (Rectified Linear Unit)**: $ f(x) = \max(0, x) $
- **Sigmoid**: $ f(x) = \frac{1}{1 + e^{-x}} $
- **TanH (Tangente Hiperbólica)**: $ f(x) = \frac{e^x - e^{-x}}{e^x + e^{-x}} $
- **Softmax**: usada na camada de saída para problemas de classificação.


## 5. Implementação Prática

Agora que entendemos a teoria, podemos implementar uma rede neural simples para classificar imagens do dataset MNIST. O modelo terá uma camada de entrada, uma camada oculta com ativação ReLU, e uma camada de saída com ativação Softmax.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Funções de ativação
def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return (x > 0).astype(float)

def softmax(x):
    exps = np.exp(x - np.max(x))  # Evita overflow
    return exps / np.sum(exps, axis=1, keepdims=True)

# Função de perda (cross-entropy)
def cross_entropy_loss(y_true, y_pred):
    m = y_true.shape[0]
    loss = -np.sum(y_true * np.log(y_pred + 1e-8)) / m  # Adiciona epsilon para evitar log(0)
    return loss

# Derivada da cross-entropy em relação à saída da softmax
def cross_entropy_derivative(y_true, y_pred):
    return y_pred - y_true

# Inicialização dos pesos
def initialize_weights(input_size, hidden_size, output_size):
    W1 = np.random.randn(input_size, hidden_size) * 0.01  # Pesos da camada oculta
    b1 = np.zeros((1, hidden_size))  # Viés da camada oculta
    W2 = np.random.randn(hidden_size, output_size) * 0.01  # Pesos da camada de saída
    b2 = np.zeros((1, output_size))  # Viés da camada de saída
    return W1, b1, W2, b2

# Propagação para frente
def forward_propagation(X, W1, b1, W2, b2):
    Z1 = np.dot(X, W1) + b1
    A1 = relu(Z1)
    Z2 = np.dot(A1, W2) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

# Backpropagation
def backward_propagation(X, Y, Z1, A1, Z2, A2, W1, W2, learning_rate):
    m = X.shape[0]

    # Cálculo do gradiente da camada de saída
    dZ2 = cross_entropy_derivative(Y, A2)
    dW2 = np.dot(A1.T, dZ2) / m
    db2 = np.sum(dZ2, axis=0, keepdims=True) / m

    # Cálculo do gradiente da camada oculta
    dA1 = np.dot(dZ2, W2.T)
    dZ1 = dA1 * relu_derivative(Z1)
    dW1 = np.dot(X.T, dZ1) / m
    db1 = np.sum(dZ1, axis=0, keepdims=True) / m

    # Atualização dos pesos
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2

    return W1, b1, W2, b2

# Transformar rótulos em one-hot encoding
def one_hot_encoding(y, num_classes):
    m = y.shape[0]
    one_hot = np.zeros((m, num_classes))
    one_hot[np.arange(m), y] = 1
    return one_hot


In [ ]:

# Treinamento da Rede Neural
def train(X_train, Y_train, X_test, Y_test, input_size, hidden_size, output_size, epochs, learning_rate):
    W1, b1, W2, b2 = initialize_weights(input_size, hidden_size, output_size)

    for epoch in range(epochs):
        # Forward propagation
        Z1, A1, Z2, A2 = forward_propagation(X_train, W1, b1, W2, b2)

        # Cálculo da perda
        loss = cross_entropy_loss(Y_train, A2)

        # Backpropagation
        W1, b1, W2, b2 = backward_propagation(X_train, Y_train, Z1, A1, Z2, A2, W1, W2, learning_rate)

        # Avaliação a cada 10 épocas
        if epoch % 10 == 0:
            _, _, _, A2_test = forward_propagation(X_test, W1, b1, W2, b2)
            test_loss = cross_entropy_loss(Y_test, A2_test)
            accuracy = np.mean(np.argmax(A2_test, axis=1) == np.argmax(Y_test, axis=1))
            print(f'Época {epoch}: Perda treino = {loss:.4f}, Perda teste = {test_loss:.4f}, Acurácia teste = {accuracy:.4f}')

    return W1, b1, W2, b2

# Predição
def predict(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_propagation(X, W1, b1, W2, b2)
    return np.argmax(A2, axis=1)


In [ ]:

# Carregar o dataset MNIST
from tensorflow.keras.datasets import mnist

(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

# Preparação dos dados
X_train = X_train.reshape(X_train.shape[0], -1) / 255.0  # Normaliza e achata as imagens
X_test = X_test.reshape(X_test.shape[0], -1) / 255.0

Y_train = one_hot_encoding(Y_train, 10)  # Converter para one-hot encoding
Y_test = one_hot_encoding(Y_test, 10)

# Definir hiperparâmetros
input_size = 784  # 28x28 pixels
hidden_size = 128
output_size = 10   # 10 classes (0 a 9)
epochs = 50
learning_rate = 0.1

# Treinar a rede neural
W1, b1, W2, b2 = train(X_train, Y_train, X_test, Y_test, input_size, hidden_size, output_size, epochs, learning_rate)

# Testar com um exemplo
index = 0
plt.imshow(X_test[index].reshape(28, 28), cmap='gray')
plt.show()

predicted_label = predict(X_test[index:index+1], W1, b1, W2, b2)
print(f'Classe predita: {predicted_label[0]}')


# Fazendo na mão



## **Passo 1: Definir a Função de Custo**

A função de custo que usamos para redes neurais de classificação binária é a **Entropia Cruzada (Binary Cross-Entropy)**:

$$
J = - \left[ y \log(\hat{y}) + (1 - y) \log(1 - \hat{y}) \right]
$$

onde:
- $ y $ é o rótulo real da amostra (0 ou 1).
- $ \hat{y} = A_3 $ é a saída da rede neural (previsão).

Se considerarmos que a **saída real** para este exemplo é $ y = 1 $, então a perda é:

$$
J = - \log(0.765) \approx 0.268
$$

Agora, usamos o **Gradiente Descendente** para atualizar os pesos.


## **Passo 2: Cálculo do Gradiente para a Camada de Saída**

A derivada da função de custo em relação à saída da rede $ A_3 $:

$$
\frac{\partial J}{\partial A_3} = - \frac{y}{A_3} + \frac{1 - y}{1 - A_3}
$$

Substituindo $ y = 1 $ e $ A_3 = 0.765 $:

$$
\frac{\partial J}{\partial A_3} = - \frac{1}{0.765} + 0 = -1.307
$$

A derivada da **função sigmoide** $ A_3 = \sigma(Z_3) $ é:

$$
\sigma'(Z_3) = A_3 (1 - A_3)
$$

$$
\sigma'(1.176) = 0.765 \times (1 - 0.765) = 0.1798
$$

Usamos a **regra da cadeia** para calcular a derivada em relação a $ Z_3 $:

$$
\frac{\partial J}{\partial Z_3} = \frac{\partial J}{\partial A_3} \times \frac{\partial A_3}{\partial Z_3}
$$

$$
= -1.307 \times 0.1798 = -0.235
$$

Agora, calculamos os gradientes para os pesos $ W_3 $ e viés $ b_3 $:

$$
\frac{\partial J}{\partial W_3} = A_2^T \times \frac{\partial J}{\partial Z_3}
$$

$$
\frac{\partial J}{\partial b_3} = \frac{\partial J}{\partial Z_3}
$$

Substituindo $ A_2 = [0.367, 0, 1.275] $:

$$
\frac{\partial J}{\partial W_3} = \begin{bmatrix} 0.367 \\ 0 \\ 1.275 \end{bmatrix} \times (-0.235)
$$

$$
= \begin{bmatrix} -0.0863 \\ 0 \\ -0.2993 \end{bmatrix}
$$

$$
\frac{\partial J}{\partial b_3} = -0.235
$$

Atualizando os pesos da camada de saída:

$$
W_3 := W_3 - \alpha \frac{\partial J}{\partial W_3}
$$

$$
b_3 := b_3 - \alpha \frac{\partial J}{\partial b_3}
$$

Se a taxa de aprendizado for $ \alpha = 0.1 $:

$$
W_3 = W_3 + \begin{bmatrix} 0.00863 \\ 0 \\ 0.02993 \end{bmatrix}
$$

$$
b_3 = b_3 + 0.0235
$$


## **Passo 3: Gradiente da Segunda Camada Oculta**

A derivada do erro em relação à saída da segunda camada $ A_2 $:

$$
\frac{\partial J}{\partial A_2} = W_3 \times \frac{\partial J}{\partial Z_3}
$$

$$
= \begin{bmatrix} 0.5 \\ -0.3 \\ 0.7 \end{bmatrix} \times (-0.235)
$$

$$
= \begin{bmatrix} -0.1175 \\ 0.0705 \\ -0.1645 \end{bmatrix}
$$

A ativação da segunda camada usa **ReLU**, então a derivada é:

$$
\frac{\partial A_2}{\partial Z_2} = \begin{bmatrix} 1 & 0 & 1 \end{bmatrix}
$$

Multiplicando:

$$
\frac{\partial J}{\partial Z_2} = \frac{\partial J}{\partial A_2} \times \frac{\partial A_2}{\partial Z_2}
$$

$$
= \begin{bmatrix} -0.1175 \\ 0 \\ -0.1645 \end{bmatrix}
$$

Atualizando os pesos:

$$
W_2 = W_2 + 0.1 \times \begin{bmatrix} 0.1645 \\ 0 \\ 0.1202 \end{bmatrix}
$$

$$
b_2 = b_2 + 0.1 \times \begin{bmatrix} 0.1175 \\ 0 \\ 0.1645 \end{bmatrix}
$$


## **Resumo Final**

1. **Cálculo da perda** (Binary Cross-Entropy).
2. **Erro da saída**: calculamos os gradientes da última camada.
3. **Propagação do erro**: ajustamos os pesos das camadas intermediárias.
4. **Atualização dos pesos** com a regra:

$$
W := W - \alpha \frac{\partial J}{\partial W}
$$

Esse processo é repetido para múltiplas amostras até que os pesos estejam otimizados.
